# notes
## intro
At a programmatic level, Spark SQL allows developers to issue ANSI SQL:2003–compatible
queries on structured data with a schema. Since its introduction in Spark 1.3,
Spark SQL has evolved into a substantial engine upon which many high-level structured
functionalities have been built. Apart from allowing you to issue SQL-like queries
on your data, the Spark SQL engine:
• Unifies Spark components and permits abstraction to DataFrames/Datasets in
Java, Scala, Python, and R, which simplifies working with structured data sets.
• Connects to the Apache Hive metastore and tables.
• Reads and writes structured data with a specific schema from structured file formats
(JSON, CSV, Text, Avro, Parquet, ORC, etc.) and converts data into temporary
tables.
• Offers an interactive Spark SQL shell for quick data exploration.
• Provides a bridge to (and from) external tools via standard database JDBC/
ODBC connectors.
• Generates optimized query plans and compact code for the JVM, for final
execution.

---------------
## The Catalyst Optimizer
The Catalyst optimizer takes a computational query and converts it into an execution
plan. It goes through four transformational phases, as shown in Figure 3-4:
1. Analysis
2. Logical optimization
3. Physical planning
4. Code generation

To see the different stages the Python code goes through, you can use the
count_mnm_df.explain(True) method on the DataFrame. Or, to get a look at the different
logical and physical plans, in Scala you can call df.queryExecution.logical
or df.queryExecution.optimizedPlan.

## Using Spark SQL in Spark Applications
The SparkSession, introduced in Spark 2.0, provides a unified entry point for programming
Spark with the Structured APIs. You can use a SparkSession to access
Spark functionality: just import the class and create an instance in your code.
To issue any SQL query, use the sql() method on the SparkSession instance, spark,
such as spark.sql("SELECT * FROM myTableName").

# refs
https://stackoverflow.com/questions/56895707/pyspark-difference-performance-for-spark-read-formatcsv-vs-spark-read-csv

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('spark-sql').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/06 00:06:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import os

flight_delay_ds_path = os.path.join(os.getcwd(),"datasets/departuredelays.csv")
print(flight_delay_ds_path)
flight_delay_df = spark.read\
                    .format("csv")\
                    .option("inferSchema","true")\
                    .option("header","true")\
                    .load(flight_delay_ds_path)

/Users/pvasud669@apac.comcast.com/repos/learnings/spark/datasets/departuredelays.csv


In [4]:
# flight_delay_df.schema
flight_delay_df.show(truncate=False)

# infer schema caused the date to be int but we need it as string


from pyspark.sql.functions import cast, col

flight_delay_df = flight_delay_df.withColumn('date',col('date').cast("string"))
flight_delay_df.schema

+-------+-----+--------+------+-----------+
|date   |delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|6    |602     |ABE   |ATL        |
|1020600|-8   |369     |ABE   |DTW        |
|1021245|-2   |602     |ABE   |ATL        |
|1020605|-4   |602     |ABE   |ATL        |
|1031245|-4   |602     |ABE   |ATL        |
|1030605|0    |602     |ABE   |ATL        |
|1041243|10   |602     |ABE   |ATL        |
|1040605|28   |602     |ABE   |ATL        |
|1051245|88   |602     |ABE   |ATL        |
|1050605|9    |602     |ABE   |ATL        |
|1061215|-6   |602     |ABE   |ATL        |
|1061725|69   |602     |ABE   |ATL        |
|1061230|0    |369     |ABE   |DTW        |
|1060625|-3   |602     |ABE   |ATL        |
|1070600|0    |369     |ABE   |DTW        |
|1071725|0    |602     |ABE   |ATL        |
|1071230|0    |369     |ABE   |DTW        |
|1070625|0    |602     |ABE   |ATL        |
|1071219|0    |569     |ABE   |ORD        |
|1080600|0    |369     |ABE   |D

StructType([StructField('date', StringType(), True), StructField('delay', IntegerType(), True), StructField('distance', IntegerType(), True), StructField('origin', StringType(), True), StructField('destination', StringType(), True)])

In [5]:
# create temp view

flight_delay_df.createOrReplaceTempView("flight_delay_view")

In [6]:
spark.sql("""
        select distinct origin, destination, distance from flight_delay_view 
        where distance > 1000 
        order by distance desc""").show()

[Stage 3:>                                                          (0 + 8) / 8]

+------+-----------+--------+
|origin|destination|distance|
+------+-----------+--------+
|   HNL|        JFK|    4330|
|   JFK|        HNL|    4330|
|   EWR|        HNL|    4312|
|   HNL|        EWR|    4312|
|   HNL|        IAD|    4186|
|   IAD|        HNL|    4186|
|   ATL|        HNL|    3912|
|   HNL|        ATL|    3912|
|   HNL|        ORD|    3687|
|   ORD|        HNL|    3687|
|   IAH|        HNL|    3392|
|   HNL|        IAH|    3392|
|   HNL|        GUM|    3303|
|   GUM|        HNL|    3303|
|   HNL|        DFW|    3288|
|   DFW|        HNL|    3288|
|   DFW|        OGG|    3224|
|   OGG|        DFW|    3224|
|   DEN|        LIH|    2967|
|   LIH|        DEN|    2967|
+------+-----------+--------+
only showing top 20 rows



In [7]:
# find all flights between San Francisco (SFO) and Chicago (ORD) with at least a two-hour delay

spark.sql("""select date, origin, destination, delay from flight_delay_view
            where origin="SFO" and destination = "ORD" and delay >= 120
            order by delay desc""").show()

+-------+------+-----------+-----+
|   date|origin|destination|delay|
+-------+------+-----------+-----+
|2190925|   SFO|        ORD| 1638|
|1031755|   SFO|        ORD|  396|
|1022330|   SFO|        ORD|  326|
|1051205|   SFO|        ORD|  320|
|1190925|   SFO|        ORD|  297|
|2171115|   SFO|        ORD|  296|
|1071040|   SFO|        ORD|  279|
|1051550|   SFO|        ORD|  274|
|3120730|   SFO|        ORD|  266|
|1261104|   SFO|        ORD|  258|
|1161210|   SFO|        ORD|  225|
|2091800|   SFO|        ORD|  223|
|1221040|   SFO|        ORD|  215|
|3121155|   SFO|        ORD|  203|
|2111256|   SFO|        ORD|  197|
|3311405|   SFO|        ORD|  196|
|1031920|   SFO|        ORD|  193|
|1021410|   SFO|        ORD|  190|
|3171215|   SFO|        ORD|  189|
|1101410|   SFO|        ORD|  184|
+-------+------+-----------+-----+
only showing top 20 rows



In [ ]:
# As an exercise, convert the date column into a readable format and find
# the days or months when these delays were most common. Were the delays related to
# winter months or holidays?)

from pyspark.sql.functions import to_date, to_timestamp, concat, lit, when, length

flight_delay_df = flight_delay_df.withColumn('date_formatted', to_timestamp(when(length('date') == 8, (concat(lit('2024'),col('date'))))\
                                                                .otherwise((concat(lit('20240'),col('date')))), 'yyyyMMddHHmm'))

In [ ]:
flight_delay_df.show()

In [ ]:
flight_delay_df.createOrReplaceTempView("flight_delay_formatted_date")

In [ ]:
spark.sql("""select month(date_formatted), day(date_formatted), count(1) as num_of_delays, (sum(delay)/60) total_delayed_hrs from flight_delay_formatted_date
            where delay > 0
            group by month(date_formatted), day(date_formatted)
            order by 3 desc""").show()

In [ ]:
# Let’s try a more complicated query where we use the CASE clause in SQL. In the following
# example, we want to label all US flights, regardless of origin and destination,
# with an indication of the delays they experienced: Very Long Delays (> 6 hours),
# Long Delays (2–6 hours), etc. We’ll add these human-readable labels in a new column
# called Flight_Delays:

spark.sql("""select origin, destination, case when delay/60 > 6 then 'very long delay'
                                                when delay/60 between 2 and 6 then 'long delay'
                                                when delay < 0 then 'arrived early'
                                                else 'delay' end as Flight_Delays from flight_delay_formatted_date""").show()

spark.sql("""select case when delay/60 > 6 then 'very long delay'
                                                when delay/60 between 2 and 6 then 'long delay'
                                                when delay < 0 then 'arrived early'
                                                else 'delay' end as Flight_Delays, count(1) from flight_delay_formatted_date
                                                group by Flight_Delays order by 2 desc""").show()

# SQL Tables and Views
Tables hold data. Associated with each table in Spark is its relevant metadata, which is
information about the table and its data: the schema, description, table name, database
name, column names, partitions, physical location where the actual data resides,
etc. All of this is stored in a central metastore.

Instead of having a separate metastore for Spark tables, Spark by default uses the
Apache Hive metastore, located at /user/hive/warehouse, to persist all the metadata
about your tables. However, you may change the default location by setting the Spark
config variable spark.sql.warehouse.dir to another location, which can be set to a
local or external distributed storage.

# Managed Versus UnmanagedTables
Spark allows you to create two types of tables: managed and unmanaged. For a managed
table, Spark manages both the metadata and the data in the file store. This could
be a local filesystem, HDFS, or an object store such as Amazon S3 or Azure Blob. For
an unmanaged table, Spark only manages the metadata, while you manage the data
yourself in an external data source such as Cassandra.
With a managed table, because Spark manages everything, a SQL command such as
DROP TABLE table_name deletes both the metadata and the data. With an unmanaged
table, the same command will delete only the metadata, not the actual data. We will
look at some examples of how to create managed and unmanaged tables in the next
section.

# Creating SQL Databases and Tables
Tables reside within a database. By default, Spark creates tables under the default
database. To create your own database name, you can issue a SQL command from
your Spark application or notebook. Using the US flight delays data set, let’s create
both a managed and an unmanaged table. To begin, we’ll create a database called
learn_spark_db and tell Spark we want to use that database:

// In Scala/Python
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

From this point, any commands we issue in our application to create tables will result
in the tables being created in this database and residing under the database name
learn_spark_db.

In [8]:
spark.sql("CREATE DATABASE learn_spark_db")

DataFrame[]

In [25]:
spark.sql("USE learn_spark_db")

DataFrame[]

In [10]:
spark.sql("CREATE TABLE managed_us_delay_flights_tbl (date STRING, delay INT, distance INT, origin STRING, destination STRING)")

25/04/06 00:09:29 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


AnalysisException: [NOT_SUPPORTED_COMMAND_WITHOUT_HIVE_SUPPORT] CREATE Hive TABLE (AS SELECT) is not supported, if you want to enable it, please set "spark.sql.catalogImplementation" to "hive".;
'CreateTable `spark_catalog`.`learn_spark_db`.`managed_us_delay_flights_tbl`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, ErrorIfExists


In [11]:
schema="date STRING, delay INT, distance INT, origin STRING, destination STRING"
flights_df = spark.read.csv(flight_delay_ds_path, schema=schema)
flights_df.write.saveAsTable("managed_us_delay_flights_tbl_new", mode="overwrite")

25/04/06 00:09:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [20]:
!pip install ipython-sql


Looking in indexes: http://localhost:3141/databee/dev/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 671.9 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 KB 7.0 MB/s eta 0:00:00
You should consider upgrading via the '/Users/pvasud669@apac.comcast.com/spark/bin/python -m pip install --upgrade pip' command.


In [22]:
%load_ext sql

 

In [27]:
spark.sql("select * from managed_us_delay_flights_tbl_new;").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01201755|    0|     449|   ORF|        ATL|
|01201610|   52|     449|   ORF|        ATL|
|01201441|    0|     449|   ORF|        ATL|
|01211755|  -15|     449|   ORF|        ATL|
|01210941|   -5|     449|   ORF|        ATL|
|01210700|    5|     449|   ORF|        ATL|
|01211243|    2|     449|   ORF|        ATL|
|01210540|   -5|     449|   ORF|        ATL|
|01211610|   27|     449|   ORF|        ATL|
|01211441|   -6|     449|   ORF|        ATL|
|01221755|   -4|     449|   ORF|        ATL|
|01220941|   -5|     449|   ORF|        ATL|
|01220700|   -2|     449|   ORF|        ATL|
|01221243|    0|     449|   ORF|        ATL|
|01220540|   -5|     449|   ORF|        ATL|
|01221610|   -5|     449|   ORF|        ATL|
|01221441|   -2|     449|   ORF|        ATL|
|01231755|   -1|     449|   ORF|        ATL|
|01230941|   -8|     449|   ORF|        ATL|
|01230700|

In [ ]:
# # Creating an unmanaged table
# # By contrast, you can create unmanaged tables from your own data sources—say, Parquet,
# # CSV, or JSON files stored in a file store accessible to your Spark application.
# # To create an unmanaged table from a data source such as a CSV file, in SQL use:
# spark.sql("""CREATE TABLE us_delay_flights_tbl(date STRING, delay INT,
# distance INT, origin STRING, destination STRING)
# USING csv OPTIONS (PATH
# '/Users/pvasud669@apac.comcast.com/repos/learnings/spark/datasets/departuredelays.csv')""")


In [ ]:
# # Creating an unmanaged table
# # By contrast, you can create unmanaged tables from your own data sources—say, Parquet,
# # CSV, or JSON files stored in a file store accessible to your Spark application.
# # To create an unmanaged table from a data source such as a CSV file, in SQL use:
# spark.sql("""CREATE TABLE us_delay_flights_tbl(date STRING, delay INT,
# distance INT, origin STRING, destination STRING)
# USING csv OPTIONS (PATH
# '/Users/pvasud669@apac.comcast.com/repos/learnings/spark/datasets/departuredelays.csv')""")


In [28]:
# And within the DataFrame API use:
# data is stored at external path and not in spark data warehouse

(flights_df
.write
.option("path", "/Users/pvasud669@apac.comcast.com/repos/learnings/spark/datasets/unmanaged-tables")
.mode("overwrite")
.saveAsTable("unmanaged_delay_flights_tbl"))


25/04/06 00:21:03 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [29]:
# querying from unmanaged table

spark.sql("select * from unmanaged_delay_flights_tbl;").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01201755|    0|     449|   ORF|        ATL|
|01201610|   52|     449|   ORF|        ATL|
|01201441|    0|     449|   ORF|        ATL|
|01211755|  -15|     449|   ORF|        ATL|
|01210941|   -5|     449|   ORF|        ATL|
|01210700|    5|     449|   ORF|        ATL|
|01211243|    2|     449|   ORF|        ATL|
|01210540|   -5|     449|   ORF|        ATL|
|01211610|   27|     449|   ORF|        ATL|
|01211441|   -6|     449|   ORF|        ATL|
|01221755|   -4|     449|   ORF|        ATL|
|01220941|   -5|     449|   ORF|        ATL|
|01220700|   -2|     449|   ORF|        ATL|
|01221243|    0|     449|   ORF|        ATL|
|01220540|   -5|     449|   ORF|        ATL|
|01221610|   -5|     449|   ORF|        ATL|
|01221441|   -2|     449|   ORF|        ATL|
|01231755|   -1|     449|   ORF|        ATL|
|01230941|   -8|     449|   ORF|        ATL|
|01230700|

# Creating Views
In addition to creating tables, Spark can create views on top of existing tables. Views
can be global (visible across all SparkSessions on a given cluster) or session-scoped
(visible only to a single SparkSession), and they are temporary: they disappear after
your Spark application terminates.
Creating views has a similar syntax to creating tables within a database. Once you create
a view, you can query it as you would a table. The difference between a view and a
table is that views don’t actually hold the data; tables persist after your Spark application
terminates, but views disappear.
You can create a view from an existing table using SQL. For example, if you wish to
work on only the subset of the US flight delays data set with origin airports of New
York (JFK) and San Francisco (SFO), the following queries will create global temporary
and temporary views consisting of just that slice of the table:

In [ ]:
#-- In SQL

# CREATE OR REPLACE GLOBAL TEMP VIEW us_origin_airport_SFO_global_tmp_view AS
# SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE
# origin = 'SFO';

# CREATE OR REPLACE TEMP VIEW us_origin_airport_JFK_tmp_view AS
# SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE
# origin = 'JFK'

In [ ]:
# You can accomplish the same thing with the DataFrame API as follows:
# In Python

df_sfo = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'SFO'")
df_jfk = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'JFK'")

# Create a temporary and global temporary view
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")

In [ ]:
# # Once you’ve created these views, you can issue queries against them just as you would
# # against a table. Keep in mind that when accessing a global temporary view you must
# # use the prefix global_temp.<view_name>, because Spark creates global temporary
# # views in a global temporary database called global_temp. For example:
# # -- In SQL

# SELECT * FROM global_temp.us_origin_airport_SFO_global_tmp_view


In [ ]:
# By contrast, you can access the normal temporary view without the global_temp
# prefix:
# -- In SQL
# SELECT * FROM us_origin_airport_JFK_tmp_view

# // In Scala/Python
# spark.read.table("us_origin_airport_JFK_tmp_view")

# // Or
# spark.sql("SELECT * FROM us_origin_airport_JFK_tmp_view")

# You can also drop a view just like you would a table:
# -- In SQL
# DROP VIEW IF EXISTS us_origin_airport_SFO_global_tmp_view;
# DROP VIEW IF EXISTS us_origin_airport_JFK_tmp_view

# // In Scala/Python
# spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")
# spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")

# Viewing the Metadata
As mentioned previously, Spark manages the metadata associated with each managed
or unmanaged table. This is captured in the Catalog, a high-level abstraction in
Spark SQL for storing metadata. The Catalog’s functionality was expanded in Spark
2.x with new public methods enabling you to examine the metadata associated with
your databases, tables, and views. Spark 3.0 extends it to use external catalog (which
we briefly discuss in Chapter 12).

In [37]:
# For example, within a Spark application, after creating the SparkSession variable
# spark, you can access all the stored metadata through methods like these:

# %config InteractiveShell.ast_node_interactivity = 'all'

# // In Scala/Python
(spark.catalog.listDatabases())
(spark.catalog.listTables())
(spark.catalog.listColumns("managed_us_delay_flights_tbl_new"))
(spark.catalog.listColumns("unmanaged_delay_flights_tbl"))

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/Users/pvasud669@apac.comcast.com/repos/learnings/spark/spark-warehouse'),
 Database(name='learn_spark_db', catalog='spark_catalog', description='', locationUri='file:/Users/pvasud669@apac.comcast.com/repos/learnings/spark/spark-warehouse/learn_spark_db.db')]

[Table(name='managed_us_delay_flights_tbl_new', catalog='spark_catalog', namespace=['learn_spark_db'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='unmanaged_delay_flights_tbl', catalog='spark_catalog', namespace=['learn_spark_db'], description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='flight_delay_view', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

[Column(name='date', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

[Column(name='date', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

# Caching SQL Tables
Although we will discuss table caching strategies in the next chapter, it’s worth mentioning
here that, like DataFrames, you can cache and uncache SQL tables and views.
In Spark 3.0, in addition to other options, you can specify a table as LAZY, meaning
that it should only be cached when it is first used instead of immediately:

-- In SQL
CACHE [LAZY] TABLE <table-name>
UNCACHE TABLE <table-name>

# Reading Tables into DataFrames
Often, data engineers build data pipelines as part of their regular data ingestion and
ETL processes. They populate Spark SQL databases and tables with cleansed data for
consumption by applications downstream.
Let’s assume you have an existing database, learn_spark_db, and table,
us_delay_flights_tbl, ready for use. Instead of reading from an external JSON file,
you can simply use SQL to query the table and assign the returned result to a
DataFrame:

# In Python
us_flights_df = spark.sql("SELECT * FROM us_delay_flights_tbl")
us_flights_df2 = spark.table("us_delay_flights_tbl")

Now you have a cleansed DataFrame read from an existing Spark SQL table. You can
also read data in other formats using Spark’s built-in data sources, giving you the flexibility
to interact with various common file formats.